In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install timm

In [ ]:
# Necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import ConcatDataset
import time
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
import copy
import glob
import timm
from torch.utils.tensorboard import SummaryWriter

In [ ]:
df = pd.read_csv("/kaggle/input/cats-in-the-wild-image-classification/WILDCATS.CSV")

In [ ]:
df.head()

In [ ]:
# Data Visualization

print(df['labels'].value_counts())
print(df['labels'].value_counts().sum())
plt.figure(figsize=(8,5))
plt.xticks(rotation=90, fontsize=10)
plt.yticks(fontsize=10)
sns.countplot(x='labels', data=df, palette='Paired')
plt.show()

In [ ]:
# Data augmentation and data definitions

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        transforms.GaussianBlur(kernel_size=(5,5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test' : transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
}

data_dir = '/kaggle/input/cats-in-the-wild-image-classification'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'valid','test']}

# Split the 'train' dataset into train and validation sets

train_data, val_data = train_test_split(image_datasets['train'], test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_data, test_size=0.5, random_state=42)


# Create the dataloaders for training, validation, and testing
dataloaders = {'train': torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4),
               'valid': torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4),
               'test' : torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)
              }

dataset_sizes = {'train': len(train_data), 'valid': len(val_data), 'test': len(test_data)}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(class_names)

In [ ]:
print("Size of training data is:")
print(len(train_data))
print('-' * 25)
print("Size of validation data is:")
print(len(val_data))
print('-' * 25)
print("Size of test data is:")
print(len(test_data))

In [ ]:
# Few images to visualize

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.axis('off') # remove axis ticks and labels


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))


# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

# Get class labels
label_names = [class_names[x] for x in classes]

# Display images with labels in a row
fig, axs = plt.subplots(1, 10, figsize=(15, 10))

for i in range(10):
    axs[i].imshow(inputs[i].permute(1, 2, 0))
    axs[i].set_title(label_names[i],fontsize=8)
    axs[i].axis('off')
    
plt.show()

In [ ]:
# Training
from torch.utils.tensorboard import SummaryWriter

def train_model(model, criterion, optimizer, scheduler, num_epochs=5, log_dir='./logs'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0


    # Create a TensorBoard summary writer
    writer = SummaryWriter(log_dir)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Log the loss and accuracy to TensorBoard
            writer.add_scalar(f'{phase}/loss', epoch_loss, epoch)
            writer.add_scalar(f'{phase}/accuracy', epoch_acc, epoch)

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    # Close the TensorBoard writer
    writer.close()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
# Visualizing predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['valid']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
# Load pre-trained Vision Transformer model
model_ft = timm.create_model('vit_base_patch16_224', pretrained=True)

# Freeze all layers in the model
for param in model_ft.parameters():
    param.requires_grad = False

# Replace the classifier with a new one that is trainable
num_ftrs = model_ft.head.in_features
model_ft.head = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),   # To prevent training data to overfit 
    nn.Linear(512, len(class_names))
)

# Move the model to the device (e.g. GPU)
model_ft = model_ft.to(device)

# Set up loss function
criterion = nn.CrossEntropyLoss()

# Set up optimizer (unfreeze the classifier's parameters)
optimizer_ft = optim.Adam(model_ft.head.parameters(), lr=0.001)

# Set up learning rate scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.25)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=12)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [ ]:
# Few Predictions
visualize_model(model_ft)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for images,labels in dataloaders['test']:
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = model_ft(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

In [ ]:
class_names = ['AFRICAN LEOPARD', 'CARACAL', 'CHEETAH', 'CLOUDED LEOPARD', 'JAGUAR', 'LIONS', 'OCELOT', 'PUMA', 'SNOW LEOPARD', 'TIGER']

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in class_names}
total_pred = {classname: 0 for classname in class_names}

# again no gradients needed
with torch.no_grad():
    for images,labels in dataloaders['test']:
        images, labels = images.cuda(), labels.cuda()
        outputs = model_ft(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[class_names[label]] += 1
            total_pred[class_names[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
